In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
import sys
from datetime import datetime

drive_project_root = '/content/drive/MyDrive/#fastcampus'
sys.path.append(drive_project_root)
!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

Mounted at /content/drive
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
     |████████████████████████████████| 813 kB 41.4 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 144 kB 54.4 MB/s 
     |████████████████████████████████| 1.7 MB 39.6 MB/s 
     |████████████████████████████████| 7.6 MB 33.0 MB/s 
     |████████████████████████████████| 22.1 MB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 39.0 MB/s 
     |████████████████████████████████| 332 kB 51.7 MB/s 
     |████████████████████████████████| 829 kB 53.0 MB/s 
     |████████████████████████████████| 636 kB 51.0 MB/s 
     |████████████████████████████████| 132 kB 44.1 MB/s 
     |████████████████████████████████| 74 kB 3.5 MB/s 
     |████████████████████████████████| 112 kB 52.7 MB/s 
     |████████████████████████████████| 180 kB 49.1 MB/s 
     |████████████████████████████████| 140 kB 50.8 MB/s 
     |████████████████████████████████| 189 kB 53

In [2]:
gpu_info = !nvidia-smi
gpu_info = "\n".join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execurte this cell.')
else:
    print(gpu_info)

Wed Dec 22 05:24:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# For data loading
from typing import List
from typing import Dict
from typing import Union
from typing import Any
from typing import Optional
from typing import Iterable
from typing import Callable
from abc import abstractmethod
from abc import ABC
from datetime import datetime
from functools import partial
from collections import Counter
from collections import OrderedDict
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pprint import pprint

from torchtext import data
from torchtext import datasets
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import Vocab, build_vocab_from_iterator, vocab
import spacy

# For configuration
from omegaconf import DictConfig
from omegaconf import OmegaConf
import hydra
from hydra.core.config_store import ConfigStore

# For logger
from torch.utils.tensorboard import SummaryWriter
import wandb
os.environ["WANDB_START_METHOD"]="thread"

In [4]:
from data_utils import dataset_split
from config_utils import flatten_dict
from config_utils import register_config
from config_utils import configure_optimizers_from_cfg
from config_utils import get_loggers
from config_utils import get_callbacks

In [5]:
# download spacy data.
!python -m spacy download en
!python -m spacy download en_core_web_sm
!python -m spacy download de
!python -m spacy download de_core_news_sm


     |████████████████████████████████| 12.0 MB 13.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 12.0 MB 8.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.9 MB 13.2 MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=fcf7f3220b88d30af91953cd4ce57ffaa96b1256d5f104a91fc5390f15ae9290
  Stored in directory: /tmp/pip-ephem-wheel-cache-5qqlkd9u/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking succ

In [6]:
data_spacy_de_en_cfg = {
    "name": "spacy_de_en",
    "data_root": os.path.join(os.getcwd(), "data"),
    'tokenizer': "spacy",
    "src_lang": "de",
    "tgt_lang": "en",
    "src_index": 0,
    "tgt_index": 1,
    "vocab": {
        "special_symbol2index": {
            "<unk>": 0,
            "<pad>": 1,
            "<bos>": 2,
            "<eos>": 3,
            
        },
        "special_first": True,
        "min_freq": 2
    }
}

data_cfg = OmegaConf.create(data_spacy_de_en_cfg)
# print(OmegaConf.to_yaml(data_cfg))

In [7]:
train_data, vaild_data, test_data = Multi30k(data_cfg.data_root)
test_data = to_map_style_dataset(test_data)

training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.66MB/s]
validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 2.10MB/s]
mmt16_task1_test.tar.gz: 100%|██████████| 43.9k/43.9k [00:00<00:00, 1.83MB/s]


In [8]:
for i in test_data:
    print(i)
    break

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.\n', 'A man in an orange hat starring at something.\n')


In [9]:
# 1. token_transform (token ...)

def get_token_transform(data_cfg: DictConfig) -> dict:
    token_transform: dict = {}
    token_transform[data_cfg.src_lang] = get_tokenizer(
        data_cfg.tokenizer, language=data_cfg.src_lang
    )
    token_transform[data_cfg.tgt_lang] = get_tokenizer(
        data_cfg.tokenizer, language=data_cfg.tgt_lang
    )
    return token_transform

token_transform = get_token_transform(data_cfg)

In [10]:
# 2. vocab_transform
def yield_tokens(
    data_iter: Iterable, lang: str, lang2index: Dict[str, int],
) -> List[str]:
    """help function to yield list of tokens"""
    for data_sample in data_iter:
        yield token_transform[lang](data_sample[lang2index[lang]])


def get_vocab_transform(data_cfg: DictConfig) -> dict:
    vocab_transform: dict = {}
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        train_iter = Multi30k(
            split="train", language_pair=(data_cfg.src_lang, data_cfg.tgt_lang)

        )
        
        # create torchtext's Vocab object
        vocab_transform[ln] = build_vocab_from_iterator(
            yield_tokens(
                train_iter,
                ln,
                {
                    data_cfg.src_lang: data_cfg.src_index,
                    data_cfg.tgt_lang: data_cfg.tgt_index
                }
            ),
            min_freq=data_cfg.vocab.min_freq,
            specials=list(data_cfg.vocab.special_symbol2index.keys()),
            special_first=True

        )

    # set UNKNOWN as the default index. --> index가 unknown으로 return  : token이 찾아지지 않은 경우 !
    # 만양에 세팅되지 않으면, runtime error가 날 수 있다!
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        vocab_transform[ln].set_default_index(data_cfg.vocab.special_symbol2index["<unk>"])
    return vocab_transform
 
vocab_transform = get_vocab_transform(data_cfg)

training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.75MB/s]


In [11]:
print(vocab_transform["de"]["<bos>"])
print(vocab_transform["en"]["hello"], vocab_transform["en"]["world"])

2
5466 1871


In [12]:
# 3 integrated transforms
# --> text_transform: [token_transform --> vocab_transform -->torch.tensor transform]

# helper function for collate_fn
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# convert to torch.tensor with bos & eos
def tensor_transform(token_ids: List[int], bos_index: int, eos_index: int):
    return torch.cat(
        (torch.tensor([bos_index]), torch.tensor(token_ids), torch.tensor([eos_index]))
    )

# src & tgt lang language text_transforms to convert raw strings --> tensor indices
def get_text_transform(data_cfg: DictConfig):
    text_transform = {}
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        text_transform[ln] = sequential_transforms(
            token_transform[ln],
            vocab_transform[ln],
            partial(
                tensor_transform,
                bos_index=data_cfg.vocab.special_symbol2index["<bos>"],
                eos_index=data_cfg.vocab.special_symbol2index["<eos>"]
            )
        )
    return text_transform

text_transform = get_text_transform(data_cfg)

In [13]:
print(text_transform['en']("hello"))
print(text_transform['en']("hello,"))
print(text_transform['en']("hello, how"))
print(text_transform['en']("hello, how are you ?"))

tensor([   2, 5466,    3])
tensor([   2, 5466,   16,    3])
tensor([   2, 5466,   16,  890,    3])
tensor([   2, 5466,   16,  890,   18, 1329, 2471,    3])


In [14]:
# 4 collate_fn -->batch를 전처리 할까?
def collate_fn(batch, data_cfg: DictConfig):
    src_batch, tgt_batch = [], []

    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[data_cfg.src_lang](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[data_cfg.tgt_lang](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=data_cfg.vocab.special_symbol2index['<pad>'])
    tgt_batch = pad_sequence(tgt_batch, padding_value=data_cfg.vocab.special_symbol2index['<pad>'])
    return src_batch, tgt_batch

def get_collate_fn(cfg: DictConfig):
    return partial(collate_fn, data_cfg=cfg.data)

# 5 data loader
def get_multi30k_dataloader(split_mode: str, language_pair: tuple, batch_size: int, collate_fn: Callable):

    iter = Multi30k(split=split_mode, language_pair=language_pair)
    dataset = to_map_style_dataset(iter)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
    return dataloader

test_dataloader = get_multi30k_dataloader("test", (data_cfg.src_lang, data_cfg.tgt_lang), 3, collate_fn=partial(collate_fn, data_cfg=data_cfg))

mmt16_task1_test.tar.gz: 100%|██████████| 43.9k/43.9k [00:00<00:00, 2.03MB/s]


In [19]:
for i in test_dataloader:
    print(i[0])
    print(i[1].shape) # [문장길이, 배치크기]
    break

tensor([[   2,    2,    2],
        [   6,    6,    6],
        [  13, 3690,   28],
        [  11, 4018,    8],
        [   7,   87,    7],
        [ 179,   44, 2874],
        [ 109,    0, 3030],
        [   9,  121,   21],
        [  17,   29,  296],
        [  79,    7,   11],
        [   0,   53,    7],
        [   5,  328, 4946],
        [   3,    5,    5],
        [   1,    3,    3]])
torch.Size([18, 3])


In [20]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [21]:
def _text_postprocessing(res: List[str]) -> str:
    if "<eos>" in res:
        res = res[:res.index("<eos>")]
    if "<pad>" in res:
        res = res[:res.index("<pad>")]
    res = " ".join(res).replace("<bos>", "")
    return res



class BaseTranslatedLightningModule(pl.LightningModule):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.cfg = cfg
        self.loss_function = torch.nn.CrossEntropyLoss(
            ignore_index=cfg.data.vocab.special_symbol2index["<pad>"]
        )

    def configure_optimizers(self):
        self._optimizers, self._scheduler = configure_optimizers_from_cfg(
            self.cfg, self
        )
        return self._optimizers, self._scheduler

    @abstractmethod
    def forward(self, src, tgt, teacher_forcing_ratio: float):
        raise NotImplementedError()

    def _forward(self, src, tgt, mode: str, teacher_forcing_ratio: float = 0.5):
        # teacher forcing:
        # seq2seq 에서 많이 쓰인다
        # src -> tgt autoregressive 학습하면, 맨 최소는 학습을 빠르게 하지만, 미래부분 학습은 기다리기 너무 힘들다
        # 랜덤으로 미래 정보도 조금 둬서 뒤에 있는 정보도 학습이 가능하게함
        # 0.5 확룰로 teacher_forcing을 하겠다

        assert mode in ["train", "val", "test"]

        # get predictions
        # teacher_forcing 용 input -->
        tgt_inputs = tgt[:-1, :] # delete ends for teacher forcing inputs
        outputs = self(src, tgt_inputs, teacher_forcing_ratio=teacher_forcing_ratio)
        tgt_outputs = tgt[1:, :] # delete start tokens

        loss = self.loss_function(
            outputs.reshape(-1, outputs.shape[-1]), # [[batch X Seq_size], other_output_shape]
            tgt_outputs.reshape(-1),
        )

        logs_detail = {
            f"{mode}_src": src,
            f"{mode}_tgt": tgt,
            f"{mode}_results": outputs,
        }

        if mode in ["val", "test"]:
            _, tgt_results = torch.max(outputs, dim=2)

            src_texts = []
            tgt_texts = []
            res_texts = []

            # convert [L X B X others] --> [B X L X others]
            for src_i in torch.transpose(src, 0, 1).detach().cpu().numpy().tolist():
                res = vocab_transform[self.cfg.data.src_lang].lookup_tokens(src_i)
                src_texts.append(_text_postprocessing(res))

            for tgt_i in torch.transpose(tgt, 0, 1).detach().cpu().numpy().tolist():
                res = vocab_transform[self.cfg.data.tgt_lang].lookup_tokens(tgt_i)
                tgt_texts.append(_text_postprocessing(res))

            for tgt_res_i in torch.transpose(tgt_results, 0, 1).detach().cpu().numpy().tolist():
                res = vocab_transform[self.cfg.data.tgt_lang].lookup_tokens(tgt_res_i)
                res_texts.append(_text_postprocessing(res))

            text_result_summary = {
                f"{mode}_src_text": src_texts,
                f"{mode}_tgt_text": tgt_texts,
                f"{mode}_results_text": res_texts,
            }
            print(f"{self.global_step} step: \n src_text: {src_texts[0]}, \n tgt_text: {tgt_texts[0]}, \n result_text: {res_texts[0]}")
            logs_detail.update(text_result_summary)

        return {f"{mode}_loss":loss}, logs_detail

    def training_step(self, batch, batch_idx):
        src, tgt = batch[0], batch[1]
        logs, _ = self._forward(src, tgt, "train", self.cfg.model.teacher_forcing_ratio)
        self.log_dict(logs)
        logs["loss"] = logs["train_loss"]
        return logs

    def validation_step(self, batch, batch_idx):
        src, tgt = batch[0], batch[1]
        logs, logs_detail = self._forward(src, tgt, "val", 0.0)
        self.log_dict(logs)
        logs["loss"] = logs["val_loss"]
        logs.update(logs_detail)
        return logs

    def test_step(self, batch, batch_idx):
        src, tgt = batch[0], batch[1]
        logs, logs_detail = self._forward(src, tgt, "test", 0.0)
        self.log_dict(logs)
        logs["loss"] = logs["test_loss"]
        logs.update(logs_detail)
        return logs

In [22]:
# untils for initializations
def init_weights(model: Union[nn.Module, pl.LightningModule]):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [24]:
# model definition

# 1. encoder
class LSTMEncoder(nn.Module):
    def __init__(
        self,
        input_dim: int,
        embed_dim: int,
        hidden_dim: int,
        n_layers: int,
        dropout: float
    ):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers =n_layers
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

        # initialization of weights
        self.apply(init_weights)

    def forward(self, src):
        # src = [seq_len, batch_size]
        embedded = self.dropout(self.embedding(src)) # [seq_len, batch_size, emb_dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        # outputs = [src_len, batch_size, hidden_dim * n direction]
        # hidden, cell = [n layers * n directions, batch_size, hidden_dim]

        # outputs will be used from top hidden layers
        return hidden, cell

# 2. decoder
class LSTMDecoder(nn.Module):
    def __init__(
        self,
        output_dim: int,
        embed_dim: int,
        hidden_dim: int,
        n_layers: int,
        dropout: float
    ):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers =n_layers

        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input: [batch size X ...] <- start token

        # outputs = [src_len, batch_size, hidden_dim * n direction]
        # hidden, cell = [n layers * n directions, batch_size, hidden_dim]

        input = input.unsqueeze(0) # <- [1, batch_size]
        embedded = self.dropout(self.embedding(input))

        # embedding = [1, batch_size, embed_dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # output = [1, batch_size, hidden_size]
        # hidden, cell = [n layers * 1 directions, batch_size, hidden_dim]

        prediction = self.fc_out(output.squeeze(0)) # [batch_size, output_size]

        return prediction, hidden, cell

 # 3. Seq2Seq(cfg) <-- encoder + decoder
class LSTMSeq2Seq(BaseTranslatedLightningModule):
    def __init__(self, cfg: DictConfig):
        super().__init__(cfg)

        self.encoder = LSTMEncoder(**cfg.model.enc)
        self.decoder = LSTMDecoder(**cfg.model.dec)

        assert self.encoder.hidden_dim == self.decoder.hidden_dim
        assert self.encoder.n_layers == self.decoder.n_layers
         
        # paramerters 들 init.
        self.apply(init_weights)

    def forward(self, src, tgt, teacher_forcing_ratio: float = 0.5):

        # src, tgt = [seq_len(can be different), batch_size]
        # for val, test teacher forcing should be 0.0

        batch_size = tgt.shape[1]
        tgt_len = tgt.shape[0]
        tgt_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(tgt_len, batch_size, tgt_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        # start_token_input (<sos> tokens)
        input = tgt[0, :]

        for t in range(1, tgt_len):
            
            # get one cell's output
            output, hiden, cell = self.decoder(input, hidden, cell)

            # set to all outpus results
            outputs[t] = output

            # decide whether going to us teacher forcing or not.
            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)

            input = tgt[t] if teacher_force else top1

        return outputs

In [25]:
data_spacy_de_en_cfg = {
    "name": "spacy_de_en",
    "data_root": os.path.join(os.getcwd(), "data"),
    'tokenizer': "spacy",
    "src_lang": "de",
    "tgt_lang": "en",
    "src_index": 0,
    "tgt_index": 1,
    "vocab": {
        "special_symbol2index": {
            "<unk>": 0,
            "<pad>": 1,
            "<bos>": 2,
            "<eos>": 3,
            
        },
        "special_first": True,
        "min_freq": 2
    }
}

data_cfg = OmegaConf.create(data_spacy_de_en_cfg)

# get_dataset
train_data, vaild_data, test_data = Multi30k(data_cfg.data_root)

token_transform = get_token_transform(data_cfg)
vocab_transform = get_vocab_transform(data_cfg)

In [26]:
# model configs

model_translate_lstm_seq2seq_cfg = {
    "name": "LSTMSeq2Seq",
    'enc': {
        "input_dim": len(vocab_transform[data_cfg.src_lang]),
        "embed_dim": 256,
        "hidden_dim": 256,
        "n_layers": 2,
        "dropout": 0.5,
    },
    "dec": {
        "output_dim": len(vocab_transform[data_cfg.tgt_lang]),
        "embed_dim": 256,
        "hidden_dim": 256,
        "n_layers": 2,
        "dropout": 0.5,
    },
    "teacher_forcing_ratio": 0.5,
}


# opt_config
opt_cfg = {
    "optimizers": [
        {
            "name": "RAdam",
            "kwargs": {
                "lr": 1e-3,
            }
        }
    ],
    "lr_schedulers": [
        {
            "name": None,
            "kwargs": {
                "warmup_end_steps": 1000
            }
        }
    ]
}

_merged_cfg_presets = {
    "LSTM_seq2seq_de_en_translate": {
        "opt": opt_cfg,
        "data": data_spacy_de_en_cfg,
        "model": model_translate_lstm_seq2seq_cfg
    }
}

# clear config hydra instance first
hydra.core.global_hydra.GlobalHydra.instance().clear()

# register preset configs
register_config(_merged_cfg_presets)

# initialization & compose configs
hydra.initialize(config_path=None)
cfg = hydra.compose('LSTM_seq2seq_de_en_translate')

# override some cfg
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{cfg.model.name}-{cfg.data.name}"

project_root_dir = os.path.join(
    drive_project_root, "runs", "de_en_translate_tutorials"
)
save_dir = os.path.join(project_root_dir, run_name)
run_root_dir = os.path.join(project_root_dir, run_name)

# train configs
train_cfg = {
    "train_batch_size": 128,
    "val_batch_size": 32,
    "test_batch_size": 32,
    "train_val_split": [0.9, 0.1],
    "run_root_dir": run_root_dir,
    "trainer_kwargs": {
        "accelerator": "dp",
        "gpus": "0",
        "max_epochs": 50,
        "val_check_interval": 1.0,
        "log_every_n_steps": 100,
        "flush_logs_every_n_steps": 100,
    }
}

# logger configs
log_cfg = {
    "loggers": {
        "WandbLogger": {
            "project": "fastcampus_de_en_translate_tutorials",
            'name': run_name,
            "tags": ["fastcampus_de_en_translate_tutorials"],
            "save_dir": run_root_dir,
        },
        "TensorBoardLogger": {
            "save_dir": project_root_dir,
            "name": run_name,
        }
    },
    "callbacks": {
        "ModelCheckpoint": {
            "save_top_k": 3,
            "monitor": "val_loss",
            "mode": "min",
            "verbose": True,
            "dirpath": os.path.join(run_root_dir, "weights"),
            "filename": "{epoch}-{val_loss:.3f}"
        },
        "EarlyStopping": {
            "monitor": "val_loss",
            "mode": "min",
            "patience": 3,
            "verbose": True
        }
    }
}

OmegaConf.set_struct(cfg, False)
cfg.train = train_cfg
cfg.log = log_cfg
# lock config
OmegaConf.set_struct(cfg, True)
print(OmegaConf.to_yaml(cfg))

opt:
  optimizers:
  - name: RAdam
    kwargs:
      lr: 0.001
  lr_schedulers:
  - name: null
    kwargs:
      warmup_end_steps: 1000
data:
  name: spacy_de_en
  data_root: /content/data
  tokenizer: spacy
  src_lang: de
  tgt_lang: en
  src_index: 0
  tgt_index: 1
  vocab:
    special_symbol2index:
      <unk>: 0
      <pad>: 1
      <bos>: 2
      <eos>: 3
    special_first: true
    min_freq: 2
model:
  name: LSTMSeq2Seq
  enc:
    input_dim: 8015
    embed_dim: 256
    hidden_dim: 256
    n_layers: 2
    dropout: 0.5
  dec:
    output_dim: 6192
    embed_dim: 256
    hidden_dim: 256
    n_layers: 2
    dropout: 0.5
  teacher_forcing_ratio: 0.5
train:
  train_batch_size: 128
  val_batch_size: 32
  test_batch_size: 32
  train_val_split:
  - 0.9
  - 0.1
  run_root_dir: /content/drive/MyDrive/#fastcampus/runs/de_en_translate_tutorials/2021-12-22T06:22:26-LSTMSeq2Seq-spacy_de_en
  trainer_kwargs:
    accelerator: dp
    gpus: '0'
    max_epochs: 50
    val_check_interval: 1.0
    log_

In [27]:
# dataloader def
train_dataloader = get_multi30k_dataloader(
    "train",
    (data_cfg.src_lang, data_cfg.tgt_lang),
    cfg.train.train_batch_size,
    collate_fn=get_collate_fn(cfg)
)
val_dataloader = get_multi30k_dataloader(
    "valid",
    (data_cfg.src_lang, data_cfg.tgt_lang),
    cfg.train.val_batch_size,
    collate_fn=get_collate_fn(cfg)
)
test_dataloader = get_multi30k_dataloader(
    "test",
    (data_cfg.src_lang, data_cfg.tgt_lang),
    cfg.train.test_batch_size,
    collate_fn=get_collate_fn(cfg)
)

validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.94MB/s]


In [31]:
for i in train_dataloader:
    print(i[0].shape)
    print(i[1].shape)
    print(i)
    break

torch.Size([27, 128])
torch.Size([24, 128])
(tensor([[ 2,  2,  2,  ...,  2,  2,  2],
        [22, 85,  6,  ..., 22, 15, 15],
        [86, 32, 70,  ..., 47, 39, 18],
        ...,
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1]]), tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [ 20, 166,   7,  ...,  20,   7,   7],
        [ 26,  37,  62,  ...,  53,  40,  17],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]]))


In [32]:
# pl trainer def & get model

def get_pl_model(cfg: DictConfig, checkpoint_path: Optional[str] = None):

    if cfg.model.name == "LSTMSeq2Seq":
        model = LSTMSeq2Seq(cfg)
    else:
        raise NotImplementedError("Not implemented model")

    if checkpoint_path is not None:
        model = model.load_from_checkpoint(cfg, checkpoint_path=checkpoint_path)
    return model


model = None
model = get_pl_model(cfg)
print(model)

LSTMSeq2Seq(
  (loss_function): CrossEntropyLoss()
  (encoder): LSTMEncoder(
    (embedding): Embedding(8015, 256)
    (rnn): LSTM(256, 256, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): LSTMDecoder(
    (embedding): Embedding(6192, 256)
    (rnn): LSTM(256, 256, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=6192, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)


In [33]:
# pytorch lightning trainer def
logger = get_loggers(cfg)
callbacks = get_callbacks(cfg)

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir = cfg.train.run_root_dir,
    num_sanity_val_steps=2,
    **cfg.train.trainer_kwargs
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/device_parser.py:131: LightningDeprecationWarning: Parsing of the Trainer argument gpus='0' (string) will change in the future. In the current version of Lightning, this will select CUDA device with index 0, but from v1.5 it will select gpus [] (same as gpus=0 (int)).
  f"Parsing of the Trainer argument gpus='{s}' (string) will change in the future."
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [34]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

7490 step: 
 src_text:  Zwei schwarz gekleidete Männer mit einer grünen und einer roten Fliege treten vor einer Menschenmenge auf ., 
 tgt_text:  Two men in black clothes with blue and red bowties are performing in front of a crowd ., 
 result_text: <unk> men in in white and a in a with lockers a crowd a crowd


Validating: 0it [00:00, ?it/s]

7717 step: 
 src_text:  Eine Gruppe von Männern lädt <unk> auf einen Lastwagen, 
 tgt_text:  A group of men are loading cotton onto a truck, 
 result_text: <unk> group of men are working a a of .
7717 step: 
 src_text:  Kind spielt auf einem Spielplatz und hängt dabei an Stangen ., 
 tgt_text:  Child playing on a playground , hanging from bars ., 
 result_text: <unk> playing playing a playground enjoying playground . .
7717 step: 
 src_text:  Eine bunt gekleidete Frau geht an einem weißen Lastwagen vorbei , der mit Flaschen gefüllt ist ., 
 tgt_text:  A woman in a colorful outfit is walking by a white truck filled with bottles ., 
 result_text: <unk> woman in black is past a with a full <unk> <unk> .
7717 step: 
 src_text:  Zwei Kinder graben Löcher in die Erde ., 
 tgt_text:  <unk> children dig holes in the dirt ., 
 result_text: <unk> children are in the .
7717 step: 
 src_text:  Mann in einem kleinen weißen Boot auf einem See ., 
 tgt_text:  Man in a small white boat on a lake ., 
 

Monitored metric val_loss did not improve in the last 3 records. Best score: 4.078. Signaling Trainer to stop.
Epoch 33, global step 7717: val_loss was not in top 3


7717 step: 
 src_text:  Zwei schwarz gekleidete Männer mit einer grünen und einer roten Fliege treten vor einer Menschenmenge auf ., 
 tgt_text:  Two men in black clothes with blue and red bowties are performing in front of a crowd ., 
 result_text: <unk> men in in white and a in white and a a in crowd crowd crowd people
